# Implement frequency method functions
Implement following functions:
1. Calculate the derivative via extension and fftn, then use Linear Regression with $L_1$ norm
2. Use curve fitting via pytorch for $\hat{u}(t,\xi) = \hat{u}(0,\xi)exp(t\sum_j(a_ji^{|j|}\xi^j))$ to calculate coefficients.
   Note for $u_{tt} = \sum_{j}a_j u_j$ the general solution for $\hat{u}$ is  $\hat{u}(t,\xi) = C_1(\xi)exp(t\sqrt{\lambda}) + C_2(\xi)exp(-t\sqrt{\lambda})$ for $\lambda = \sum_j(a_ji^{|j|}\xi^j), C_1 + C_2 = \hat{u}(0,\xi)$
3. Use PDE-Find Algorithm



In [1]:
# Packages
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
import scipy.linalg
import statsmodels.api as sm
import itertools
import pysindy as ps

from scipy.optimize import minimize
from scipy.fft import fftn, ifftn, fftfreq,fft

### Functions for calculating derivative and extension

In [2]:
# Calculates the spectral derivative for n-dimensional input and order
# Takes ax and d as liste e.g. ax = [0], d =[0.1]
def calc_deriv_fftn(f,ax,d):
    if isinstance(ax, list) == False or isinstance(ax, list) == False:
        print("ax and d are only valid as a list")
        print("Exit calc_deriv_fftn")
        return None
    order = len(ax)
    fftf = fftn(f)
    frequencies = []
    # Calculate frequencies
    for i in range(order):
        frequencies.append(fftfreq(f.shape[ax[i]],d[i]))

    #Expand frequencies so we can  multiply
    for j in range(order):
        liste = list(range(f.ndim)) 
        liste.remove(ax[j])
        for i in liste:
            frequencies[j] = np.expand_dims(frequencies[j],axis=i)
            frequencies[j] = np.repeat(frequencies[j],f.shape[i],axis=i)
        fftf *=2*np.pi*1j*frequencies[j]
    return ifftn(fftf).real


def create_data_2d(T_start, T_end, L_x, N_t, N_x):
    t = np.linspace(T_start, T_end, num=N_t)
    x = np.linspace(-L_x/2.0, L_x/2.0, num=N_x)
    T,X = np.meshgrid(t,x)
    return T,X,t,x

def create_data_3d(T_start, T_end, L_x,L_y, N_t, N_x,N_y):
    t = np.linspace(T_start, T_end, num=N_t)
    x = np.linspace(-L_x/2.0, L_x/2.0, num=N_x)
    y = np.linspace(-L_y/2.0, L_y/2.0, num=N_y)
    T,X,Y = np.meshgrid(t,x,y)
    return T,X,Y,t,x,y

In [3]:
# This function takes f and x, creates more datapoints s.t. f 
# can be extended to a periodic function with value 0 at end and beginning
# return newf, newx, newn, only works for one_dim function
def flattenandextendfunc_1d(f,x):
    # error handling
    if f.size!=x.size:
        print("Error: f and x do not have same size: f.size="
              +str(f.size)+",x.size="+str(x.size))
        print("Exit flattenandextendfu")
        return None
    # Get necessary data to extend f
    dx = x[1]-x[0]
    A=x[0]
    B=x[x.size-1]
    n=max(int(x.size/10),20) 
    # Create new data
    newx=np.arange(A-n*dx,B +n*dx,dx)
    newf=np.empty(newx.size, dtype=float)
    oldf=newf.copy()
    # Get slope at beg and end
    slope_start = (f[1]-f[0])/dx
    slope_end = (f[f.size-1]-f[f.size-2])/dx
    # Extend oldf
    oldf[n:n+f.size]=f
    oldf[n+f.size:]=slope_end*dx*np.arange(1, oldf[n+f.size:].size+1)+f[f.size-1]
    oldf[:n]=f[0]-slope_start*dx*np.arange(1, oldf[:n].size+1)[::-1] #[::-1] means reverse array
    # Create newf: in the middle equal to f, continuous extension such that 
    # start and endpoint are equal to one
    a=30
    half = A+(B-A)/2
    newf[newx<=half]=1.0/(1.0 + np.exp(-a*(newx[newx<=half]-(A-n/2*dx))))
    newf[newx>half]=-1.0/(1.0 + np.exp(-a*(newx[newx>half]-(B+n/2*dx))))+1.0
    newf = newf*oldf
    return newf,newx,n

def getnewsize(x):
    dx = x[1]-x[0]
    A=x[0]
    B=x[x.size-1]
    n=max(int(x.size/10),20) 
    newx=np.arange(A-n*dx,B +n*dx,dx)
    return newx.size

# Extends multidimensional function
def flattenandextendfunc_nd(f,inp):
    dim = f.ndim # get dimension
    newsize=[getnewsize(myinp) for myinp in inp]
    n=[]
    newinp=inp.copy()
    temp=[]
    liste = list(range(dim))
    # Iterate through axis: extend function for selected axis
    for axis in liste:
        liste_drop = liste.copy()
        liste_drop.remove(axis)
        
        # Calculate new shape
        shape = list(f.shape)
        shape[axis]=newsize[axis]
        shape = tuple(shape)
        temp = np.zeros(shape) #Create array with new extended shape

        start = np.zeros(dim, dtype=int) #for slicing
        end = np.zeros(dim, dtype=int)+f.shape[axis] # for slicing
        newend = np.zeros(dim, dtype=int)+newsize[axis] #for slicing new array

        # Select and save ranges in rangelist for iterating
        rangelist =[]
        for otheraxis in liste_drop:
            rangelist.append(range(f.shape[otheraxis]))  

        #Extend function for axis
        for J in itertools.product(*rangelist):
            i=0
            #Calculate slice
            for otheraxis in liste_drop:
                start[otheraxis] = J[i]
                end[otheraxis]=J[i]+1
                newend[otheraxis] = end[otheraxis]
                i+=1
            myslice =tuple(slice(*indexes) for indexes in zip(start, end))
            newslice =tuple(slice(*indexes) for indexes in zip(start, newend))

            #Extend one dimensional version
            col, newinp_axis, n_axis =flattenandextendfunc_1d(f[myslice].ravel(),newinp[axis])
            temp[newslice] = col.reshape(temp[newslice].shape)

        f = temp #Set f equal to temp    
        # Save new Input and new numbers f data
        newinp[axis]=newinp_axis
        n.append(n_axis)

    return f,newinp,n

### Functions for $L_1$ Regression

In [4]:
#Define functions for L_1 regression

def check_orderinput_matches_orderaxis(inp,u):
    for i in range(u.ndim):
        if u.shape[i]!=inp[i].size:
            print("Axis "+str(i)+"  with shape "+str(u.shape[i])+
                  " does not match size of inp["+str(i)+"] "+str(inp[i].size))
            return False
        else:
            return True
        
def fit(X, params):
    return X.dot(params)

def cost_function(params, inp, y):
    return np.sum(np.abs(y - fit(inp, params)))

def frequency_method_regr(u,inp,order=1,time_deriv = 1):
    #Error prevention
    if check_orderinput_matches_orderaxis(inp,u)==False:
        return None #prevent errors in flattenandextenfunc
    
    newu,newinp,n = flattenandextendfunc_nd(u,inp)

    inp_regr = []
    inp_regr.append(u.ravel())
    d = [myinp[1]-myinp[0] for myinp in inp] #saves distance
    #Define slice to get right values from extension
    start = n
    end = [sum(x) for x in zip(n, list(u.shape))] #end = n0+u.shape[0],n1+u.shape[1],...
    myslice =tuple(slice(*indexes) for indexes in zip(start, end))
    
    #Calculate derivative for order and dimension
    for j in range(1,order+1):
            liste = list(range(0,u.ndim))
            liste.remove(1) #remove axis for t
            for subset in itertools.combinations_with_replacement(liste,j):
                axis = list(subset)
                dist = [d[ax] for ax in axis]
                x_ai = calc_deriv_fftn(newu,axis,dist)
                inp_regr.append(x_ai[myslice].ravel())

    #Define In- and Output for regression
    #Distinguish between time_deriv =1 <-> u_t = ... and time_deriv=2 <-> u_tt = ...
    if time_deriv == 1:
        y = calc_deriv_fftn(newu,[1],[d[1]])
    elif time_deriv ==2:
        y = calc_deriv_fftn(newu,[1,1],[d[1],d[1]])
    
    #If data is real:
    y=y[myslice].ravel()
    input_regr = np.stack(inp_regr,axis=1)
    #Use values from OLS regression as initial data for minimization
    model = sm.OLS(np.real(y),np.real(input_regr))
    results = model.fit()
    output = minimize(cost_function, (results.params), args=(np.real(input_regr), np.real(y)))

    return output.x

### Functions for Curvefit
Important: Input u must be given with the right periodicity!

In [5]:
#multidim input
def get_boundarypoints_time(func,t_0,t_1,N_t):
    #get V(x)=func(x,t_0), V1(x)=func(x,t_1)
    start = np.zeros(func.ndim, dtype=int) #for slicing
    end = start+func.size # for slicing
    start[1],end[1]=[t_0,t_0+1] #select t=t_0
    V = func[tuple(slice(*indexes) for indexes in zip(start, end))]
    V = np.repeat(V,N_t,axis=1)  #expand V for multiplication
    V = torch.from_numpy(V.ravel()).type(torch.complex128)
    start[1],end[1]=[t_1,t_1+1] #select t=t_1
    V1 = func[tuple(slice(*indexes) for indexes in zip(start, end))]
    V1 = np.repeat(V1,N_t,axis=1)  #expand V for multiplication
    V1 = torch.from_numpy(V1.ravel()).type(torch.complex128)
    return V,V1

def get_frequencies(d,N,ax,u):
    dimx = len(N)
    frequencies=[] 
    # Calculate frequencies
    for i in range(dimx):
        frequencies.append(fftfreq(u.shape[ax[i]],d[i]))
        
    #Expand frequencies so we can  multiply
    for j in range(dimx):
        liste = list(range(u.ndim))
        liste.remove(ax[j])
        for i in liste:
            frequencies[j] = np.expand_dims(frequencies[j],axis=i)
            frequencies[j] = np.repeat(frequencies[j],u.shape[i],axis=i) 
        frequencies[j]=torch.from_numpy(frequencies[j].ravel()).type(torch.complex128)
    return frequencies

def func(para,u_dft,N_t,T,order,dimx,frequencies,time_deriv=1):
    #Calcuclate u_dft(t=0), u_dft(t=t1)
    V,V1=get_boundarypoints_time(u_dft,0,N_t-1,N_t)
    
    #Calculate tempsum = \sum_a \xi^a * i^|a| * t
    temp_sum = (para[0][0]*torch.ones(T.shape)).type(torch.complex128) # t will be mutiplied later
    for j in range(1,order+1):
            deriv_num=0
            #subset element in combination of derivatives e.g. [0,0],[0,1],[1,1], 0=x,1=y
            for subset in itertools.combinations_with_replacement(range(0,dimx),j):
                temp_prod=1.0
                for freq_num in range(len(subset)):
                    temp_prod*= 1j*2*np.pi*frequencies[subset[freq_num]]
                temp_prod*=para[j][deriv_num]
                temp_sum+=temp_prod 
                deriv_num+=1
    shift = torch.tensor([700],dtype=torch.complex128) #to prevent overflow
    result= torch.exp(shift)*V*torch.exp(T*temp_sum-shift)
    #Note we add 1e-14 because gradient of sqrt not well defined at zero
    if time_deriv == 2:
        #a=V-b, b=(V1-V exp(sqrt(tempsum)t)/( exp(-t sqrt(tempsum))-exp(t sqrt(tempsum)) )
        a=torch.zeros(T.shape,dtype=torch.complex128)
        b=torch.zeros(T.shape,dtype=torch.complex128)
        t0=T[0:1]
        t1 = T[N_t-1:N_t]
        temp_divide=torch.exp(-t1*torch.sqrt(temp_sum+1e-14)) - torch.exp(t1*torch.sqrt(temp_sum+1e-14))
        mask =  [temp_divide!=0]
        b[mask]=(V1[mask]-V[mask]*torch.exp(t1*torch.sqrt(temp_sum[mask])))/temp_divide[mask]
        a = V-b
        result= a*torch.exp(T*torch.sqrt(temp_sum+1e-14)) + b*torch.exp(-T*torch.sqrt(temp_sum+1e-14))
    return result
    

def frequency_method_curvefit(u,T,listofXinput,ax=[0],order=1,epochs=2000, learning_rate=0.01
                                , print_loss=False, time_deriv=1):
    dtype = torch.float64
    dimx = len(listofXinput)
    d=[] #saves distanc for Xinputs
    N=[] #saves amount of Xinputs
    for inp in listofXinput:
        d.append(inp[1]-inp[0])
        N.append(inp.size)
    N_t = T.shape[1]
    u_dft=fftn(u,axes=ax)
    
    frequencies = get_frequencies(d,N,ax,u)
    
    # Set x and y
    T = torch.from_numpy(T.ravel()).type(torch.complex128)
    y = torch.from_numpy(u_dft.ravel()).type(torch.complex128)

    # Initialise Parameter
    para=torch.nn.ParameterList([torch.nn.Parameter(
        torch.rand( int(scipy.special.binom(dimx+j-1,j)), dtype=dtype, requires_grad=True))
                             for j in range(order+1)])
    #not right right nowters
    optimiser = torch.optim.Adam(para, lr = learning_rate)
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, patience = 10)
    loss_func = torch.nn.L1Loss()
    lossarray = np.array([])
    torch.autograd.set_detect_anomaly(True)#gives error if grad returns NaN
        
    for t in range(epochs):
        #Forward Pass
        y_pred = func(para,u_dft,N_t,T,order,dimx,frequencies,time_deriv)  
        #Error Prevetion
        if torch.isinf(y_pred).any():
            print("Error: inf values in y_pred caused by torch.exp")
            print("Epoch: "+str(t))
            return None
        if torch.isnan(y_pred).any():
            print("Error: NaN values in y_pred")
            print("Epoch: "+str(t))
            return None
        # Compute and save loss
        loss = loss_func(y_pred,y)
        lossarray= np.append(lossarray,loss.item())
        #Print loss
        if (t%int(epochs/5) == (int(epochs/5)-1)) & print_loss == True:
            print(t, loss.item())
            print([para[i] for i in range(len(para))])

        # Use optimiser and autograd to update weights
        optimiser.zero_grad()     
        loss.backward()
        optimiser.step()
    
    if print_loss ==True:
        plt.title("Loss")
        plt.plot(lossarray)
        #plt.yscale('log')
        plt.show()
    
    return [para[i] for i in range(len(para))]

## PDE-Find
see more on https://github.com/dynamicslab/pysindy
and https://www.science.org/doi/10.1126/sciadv.1602614

In [6]:
def pde_find(u,t,x,order,print_model=True):
    dt = t[1]-t[0]
    pde_lib = ps.PDELibrary(
        derivative_order=order,
        spatial_grid=x,
    )

    u_sindy = u.reshape(len(x), len(t), 1) # u in shape (len(x),len(t))
    optimizer = ps.STLSQ(threshold=0.0)
    model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer,feature_names=["u"])
    model.fit(u_sindy, t=dt)
    if print_model==True:
        model.print()
    return model.coefficients()
    
def pde_find_sec_timederiv(u,t,x,order,print_model=True):
    dt = t[1]-t[0]
    pde_lib = ps.PDELibrary(
        derivative_order=order,
        spatial_grid=x,
    )
    
    utt = ps.FiniteDifference(d=2, axis=1,drop_endpoints=False)._differentiate(u, dt) 
    u_sindy = u.reshape(len(x), len(t), 1) # u in shape (len(x),len(t))
    utt = utt.reshape(len(x),len(t),1) # u_tt in shape (len(x),len(t))
    optimizer = ps.STLSQ(threshold=0.0)
    model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer,feature_names=["u"])
    model.fit(u_sindy, t=dt, x_dot = utt)
    if print_model==True:
        model.print()
    return model.coefficients()